In [ ]:
!pip install pure-prng

In [ ]:
# Necessary imports
from pure_prng_package import pure_prng
import numpy as np
import tensorflow as tf
from keras.layers import *
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import math
import time



In [ ]:
# Class used to access my implmentations of PRNGs
class PRNGManagement():
  # Initialises object and sets the default seed
  def __init__(self):
      self.seed = 0

  # Method to seed the PRNG (seeds all PRNGs in class)
  def seed_PRNG(self, seed:int):
    self.seed:int = seed
    self.random_number:int = seed

  def bit_success(self, model, inputData, trueOutputs, sequence_length):
    """ 
    Method to evaluate the provided model and store the amount of
    successful predictions for each bit of the output
    :param model: keras model - Model used to generate predicitions
    :param trueOutputs: list[list[int]]- List containing the expected y outputs
    :param sequence_length: int - Length of generated binary string being predicited
    :return list[int] - Amount of successful predictions for each bit
    """
    # Set initial amount of successful predictions for each bit to zero
    successfulPredicts = [0]*sequence_length
    # Feeds the input data to the model and stores the predictions made
    predicted = (model.predict(inputData).round())
    # Iterate over all outputed data
    for testIndex in range(0, len(inputData)):
      # Iterate over each bit in output
      for i in range(sequence_length):
        # If the predicted bit matches the true bit value then increment the successful predicts for the current bit
        if predicted[testIndex][i] == trueOutputs[testIndex][i]: successfulPredicts[i] += 1
        # Prediction may be greater than 1 if the prediction is made with high certainity
        elif predicted[testIndex][i] > 1 and trueOutputs[testIndex][i] == 1: self.successfulPredicts[i] += 1

    return successfulPredicts
   

  def zero_only_PRNG(self, length=100):
    """ 
    Returns a binary string containing only 0 of specified length.
    Used to test for major flaws in models
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    return "0" * length        


  def alternating_bits_PRNG(self, length=100):
    """ 
    Returns output of a basic PRNG implementation that alernates each bit (010101)
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    # Use seed to determine the starting bit of the generated binary string
    self.seed = self.seed%2
    # Utilises efficent method to repeat a string pattern
    if (self.seed == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    # Length of generated binary string is odd
    if (length%2 == 1):
      # Add final bit to string
      output += str(self.seed)
      # Set the new seed value
      if (self.seed == 0): self.seed = 1
      else: self.seed = 0
    
    return output


  def alternating_num_PRNG(self):
    """ 
    Returns output of a basic PRNG implementation that alernates between two binary strings
    :return string - generated binary string
    """
    # Use seed to determine the binary string to be returned
    self.seed = (self.seed+1)%2
    if (self.seed == 0):
      # Convert integer to a binary string 
      randomBinary = str(bin(1643712566))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary
    else:
      # Convert integer to a binary string 
      randomBinary = str(bin(2372817037))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary


  def basic_equation_based(self, mult:int, add:int, mod:int, leng:int) -> str:
    """ 
    Returns output of a very weak equation based PRNG implementation
    Expected to be predicted near perfectly
    :return string - generated binary string
    """
    # Generates random number using previous output as seed
    self.random_number = (mult * self.random_number + add) % 2**mod
    # Converts generated number to a binary string
    bits_string = bin(self.random_number)[2:]
    # Returns binary string after using padding to ensure a length of 32
    return bits_string.zfill(leng)


  ## Different implmentations of equation based generators
  def basic_equation_based1(self) -> str:
    return self.basic_equation_based(20, 52, 32, 32)

  def basic_equation_based2(self) -> str:
    return self.basic_equation_based(36791, 83247, 32, 32)

  # Expects odd starting seed
  def poor_equation_based(self) -> str:
    return self.basic_equation_based(65539, 0, 31, 32)


PRNGHandler = PRNGManagement()

In [ ]:
# Class used to manage the PRNGs offered by the PRNG library
class PRNGLibManagement():
  def __init__(self, PRNGType:str, outputLen:int):
    self.change_PRNG(PRNGType, outputLen)

  # Change the type of the current PRNG
  def change_PRNG(self, PRNGType:str, outputLen:int):
    self.outputLen = outputLen
    self.PRNGType = PRNGType
    self.currentGen = pure_prng(int(time.time()), prng_type=PRNGType).source_random_number()

  # Seed the current PRNG
  def seed_current(self, seed=int(time.time())):
    self.currentGen = pure_prng(int(seed), prng_type=self.PRNGType).source_random_number()

  # Generates an output from the curent PRNG
  def output_current(self):
    # Converts generated number to a binary string
    bits_string = bin(next(self.currentGen))[2:]
    # Returns binary string after using padding to ensure a consistent length
    return bits_string.zfill(self.outputLen)

  # Generates output from current PRNG as a list containing the integer bits
  def next_ints(self):
    # Converts generated binary string to list of ints
    return [int(bit) for bit in self.output_current()] 

  # Get stream of binary data form current generator
  def get_stream(self, number_of_blocks:int):
    stream = []
    for _ in range(number_of_blocks):
      block = self.next_ints()
      stream.extend(block)
    return stream

In [ ]:
# Creates object to use the 'Ran64' PRNG
PRNGLibHandler = PRNGLibManagement("Ran64", 64)

In [ ]:
# Sets paramemters for generating train/test data
num_blocks = 1000
sequence_length = 64

# Number of samples must be a multiple of 100 to prevent an error
num_samples = (num_blocks-1)*sequence_length

print("Number of samples: ", num_samples)

# Seed generator
PRNGLibHandler.seed(23)
# Get stream data 
streamData = PRNGLibHandler.get_stream(num_blocks)


# Stores output in variable to allow the sample to be featued in both x and y data
X_data = []
Y_data = []

# Extracts stream data samples
for i in range(num_samples):
  X_data.append(streamData[i:i+sequence_length])
  Y_data.append([streamData[i+sequence_length]])

del streamData # saves memory

Number of samples:  12736


In [ ]:
# Sets parameters for model
epochs = 20

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.1))
  model.add(BatchNormalization())
  model.add(Dense(sequence_length//2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4160      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_2 (Batc  (None, 32)               1

In [ ]:
batch_size = 512
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/20
875/875 [==============================] - 51s 55ms/step - loss: 0.7078 - accuracy: 0.4990 - val_loss: 0.6953 - val_accuracy: 0.4984
Epoch 2/20
875/875 [==============================] - 6s 7ms/step - loss: 0.6941 - accuracy: 0.5044 - val_loss: 0.6942 - val_accuracy: 0.4994
Epoch 3/20
875/875 [==============================] - 7s 8ms/step - loss: 0.6933 - accuracy: 0.5067 - val_loss: 0.6938 - val_accuracy: 0.4991
Epoch 4/20
875/875 [==============================] - 6s 6ms/step - loss: 0.6930 - accuracy: 0.5087 - val_loss: 0.6939 - val_accuracy: 0.4979
Epoch 5/20
875/875 [==============================] - 6s 7ms/step - loss: 0.6928 - accuracy: 0.5107 - val_loss: 0.6940 - val_accuracy: 0.5002
Epoch 6/20
875/875 [==============================] - 6s 7ms/step - loss: 0.6928 - accuracy: 0.5115 - val_loss: 0.6938 - val_accuracy: 0.4988
Epoch 7/20
875/875 [==============================] - 6s 7ms/step - loss: 0.6926 - accuracy: 0.5122 - val_loss: 0.6939 - val_accuracy: 0.4998
Epoc

In [ ]:
# Sets parameters for model
epochs = 10
batch_size = 100


# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(LSTM(int(sequence_length), input_shape=(sequence_length, 1), return_sequences=True))
  model.add(LSTM(int(sequence_length/1.5), return_sequences=True))
  model.add(LSTM(int(sequence_length/2)))
  model.add(Dense(1, activation='sigmoid'))


  model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

  model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 64, 42)            17976     
                                                                 
 lstm_2 (LSTM)               (None, 32)                9600      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 44,505
Trainable params: 44,505
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
4478/4478 [==============================] - 110s 23ms/step - loss: 0.6932 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.4993
Epoch 2/10
4478/4478 [==============================] - 60s 13ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6933 - val_accuracy: 0.4993
Epoch 3/10
4478/4478 [==============================] - 61s 14ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4993
Epoch 4/10
4478/4478 [==============================] - 61s 14ms/step - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6932 - val_accuracy: 0.4993
Epoch 5/10
4478/4478 [==============================] - 62s 14ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.4993
Epoch 6/10
4478/4478 [==============================] - 62s 14ms/step - loss: 0.6932 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.4993
Epoch 7/10
4478/4478 [==============================] - 60s 13ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6931 

## Window approach with concatenated samples

In [ ]:
# Sets paramemters for generating train/test data
num_blocks = 10000
sequence_length = 64
samplesConcatenated = 3
num_samples = sequence_length*num_blocks-1-(sequence_length*samplesConcatenated)
num_samples = int(num_samples)
# Number of samples must be a multiple of 100 to prevent an error
#num_samples = int(num_samples/100)*100

print("Number of samples: ", num_samples)

Ran64Handler.seed(23)
streamData = Ran64Handler.get_stream(num_blocks)


# Stores output in variable to allow the sample to be featued in both x and y data
X_data = []
Y_data = []

for i in range(0, num_samples):
#for i in range(0, sequence_length*num_blocks-1):
  X_data.append(streamData[i:i+sequence_length*samplesConcatenated])
  Y_data.append([streamData[i+sequence_length*samplesConcatenated]])

del streamData

X_data = np.array(X_data)
# Reshape binary data to match input shape for Conv1D
X_data = np.reshape(X_data, (num_samples, sequence_length*samplesConcatenated, 1))

Y_data = np.array(Y_data)
# Reshape binary data to match input shape for Conv1D
Y_data = np.reshape(Y_data, (num_samples,))

Number of samples:  639807


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=32)
del X_data
del Y_data

In [ ]:
from keras.layers import Conv1D, Dense, Flatten
from keras.layers import *

# Sets parameters for produced model
epochs = 10
batch_size = 50

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(sequence_length*samplesConcatenated,1)))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.2))
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 190, 128)          512       
                                                                 
 flatten_4 (Flatten)         (None, 24320)             0         
                                                                 
 batch_normalization_13 (Bat  (None, 24320)            97280     
 chNormalization)                                                
                                                                 
 dropout_5 (Dropout)         (None, 24320)             0         
                                                                 
 dense_17 (Dense)            (None, 64)                1556544   
                                                                 
 batch_normalization_14 (Bat  (None, 64)               256       
 chNormalization)                                     

In [ ]:
# Sets parameters for model
epochs = 10
batch_size = 100


# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Dense(sequence_length, input_shape=(sequence_length*samplesConcatenated, ), activation='relu'))
  model.add(Dense(sequence_length//2, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                41024     
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 43,137
Trainable params: 43,137
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
8958/8958 [==============================] - 68s 7ms/step - loss: 0.7008 - accuracy: 0.4997 - val_loss: 0.6941 - val_accuracy: 0.4981
Epoch 2/10
8958/8958 [==============================] - 64s 7ms/step - loss: 0.6936 - accuracy: 0.5002 - val_loss: 0.6935 - val_accuracy: 0.5005
Epoch 3/10
8958/8958 [==============================] - 64s 7ms/step - loss: 0.6933 - accuracy: 0.5042 - val_loss: 0.6936 - val_accuracy: 0.5012
Epoch 4/10
8958/8958 [==============================] - 64s 7ms/step - loss: 0.6931 - accuracy: 0.5059 - val_loss: 0.6934 - val_accuracy: 0.5007
Epoch 5/10
8958/8958 [==============================] - 65s 7ms/step - loss: 0.6929 - accuracy: 0.5097 - val_loss: 0.6936 - val_accuracy: 0.4994
Epoch 6/10
8958/8958 [==============================] - 65s 7ms/step - loss: 0.6927 - accuracy: 0.5113 - val_loss: 0.6936 - val_accuracy: 0.5003
Epoch 7/10
8958/8958 [==============================] - 65s 7ms/step - loss: 0.6927 - accuracy: 0.5116 - val_loss: 0.6936 - val_ac

## Window approach with concatenated samples (deep learning)

In [ ]:
# Sets parameters for model
epochs = 10
batch_size = 100


# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(LSTM(int(sequence_length*2), input_shape=(sequence_length*samplesConcatenated, 1), return_sequences=True))
  model.add(LSTM(int(sequence_length), return_sequences=True))
  model.add(LSTM(int(sequence_length/2)))
  model.add(Dense(1, activation='sigmoid'))


  model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

  model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 640, 128)          66560     
                                                                 
 lstm_4 (LSTM)               (None, 640, 64)           49408     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_22 (Dense)            (None, 1)                 33        
                                                                 
Total params: 128,417
Trainable params: 128,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
2236/2236 [==============================] - 387s 170ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5013
Epoch 2/10
2236/2236 [==============================] - 192s 86ms/step - loss: 0.6932 - accuracy: 0.5014 - val_loss: 0.6932 - val_accuracy: 0.5013
Epoch 3/10
2236/2236 [==============================] - 178s 80ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.5013
Epoch 4/10
2236/2236 [==============================] - 177s 79ms/step - loss: 0.6932 - accuracy: 0.5010 - val_loss: 0.6932 - val_accuracy: 0.5013
Epoch 5/10
2236/2236 [==============================] - 178s 79ms/step - loss: 0.6932 - accuracy: 0.5007 - val_loss: 0.6931 - val_accuracy: 0.4995
Epoch 6/10
2236/2236 [==============================] - 177s 79ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.4989
Epoch 7/10
2236/2236 [==============================] - 177s 79ms/step - loss: 0.6932 - accuracy: 0.5013 - val_loss: 